<a href="https://colab.research.google.com/github/Aswaaa2001/My_Python_Projects/blob/main/Task1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import json
import pprint
from typing import List, Dict, Optional, Any
import openai
import jsonschema


In [ ]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    GROQ_API_KEY = input("Paste your Groq API key (will not be saved): ").strip()

openai.api_key = GROQ_API_KEY
openai.api_base = os.environ.get("GROQ_API_BASE", "https://api.groq.com/openai/v1")
print("API configured:", openai.api_base)

In [ ]:
class ConversationManager:
    def __init__(self, model="groq-llm-1", max_turns=None, max_chars=None, summary_interval=3):
        self.model = model
        self.max_turns = max_turns
        self.max_chars = max_chars
        self.summary_interval = summary_interval
        self.history: List[Dict[str, str]] = []
        self.summaries: List[dict] = []
        self.run_count = 0

    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})
        self.run_count += 1
        self.truncate_history()
        if self.run_count % self.summary_interval == 0:
            self.summarize_history()

    def truncate_history(self):
        if self.max_turns:
            self.history = self.history[-self.max_turns:]
        if self.max_chars:
            total = sum(len(m["content"]) for m in self.history)
            while total > self.max_chars and len(self.history) > 1:
                removed = self.history.pop(0)
                total -= len(removed["content"])

    def summarize_history(self):
        conversation_text = "\n".join(f"{m['role']}: {m['content']}" for m in self.history)

        functions = [
            {
                "name": "structured_summary",
                "description": "Summarize the conversation concisely.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "short_summary": {"type": "string"},
                        "key_points": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["short_summary"]
                }
            }
        ]

        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Summarize the conversation concisely in JSON."},
                    {"role": "user", "content": conversation_text}
                ],
                functions=functions,
                function_call={"name": "structured_summary"},
                temperature=0.2,
                max_tokens=400
            )
            payload = response.choices[0].message.get("function_call", {}).get("arguments", "{}")
            parsed = json.loads(payload)
        except Exception as e:
            parsed = {"short_summary": f"Error: {e}", "key_points": []}

        self.summaries.append(parsed)
        self.history = [{"role": "system", "content": "Summary: " + parsed["short_summary"]}]

    def show_history(self):
        return self.history

    def show_summaries(self):
        return self.summaries


In [ ]:
pp = pprint.PrettyPrinter(indent=2)

cm = ConversationManager(max_turns=4, max_chars=200, summary_interval=3)

samples = [
    ("user", "Hello assistant, how are you?"),
    ("assistant", "I’m fine, how can I help you today?"),
    ("user", "Tell me about AI."),
    ("assistant", "AI stands for Artificial Intelligence."),
    ("user", "Okay, summarize AI in one sentence."),
    ("assistant", "AI is machines performing tasks like humans.")
]

for role, msg in samples:
    print(f"\nAdding: {role} → {msg}")
    cm.add_message(role, msg)
    print("History:", cm.show_history())

print("\nSummaries:")
pp.pprint(cm.show_summaries())


In [ ]:
info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": ["string", "null"]},
        "email": {"type": ["string", "null"], "format": "email"},
        "phone": {"type": ["string", "null"]},
        "location": {"type": ["string", "null"]},
        "age": {"type": ["integer", "null"]}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

functions_info_extraction = [
    {
        "name": "extract_user_info",
        "description": "Extract structured user details from the chat.",
        "parameters": info_schema
    }
]

def extract_info_from_chat(chat_text: str, model="groq-llm-1") -> dict:
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "Extract details and return JSON according to schema."},
                {"role": "user", "content": chat_text}
            ],
            functions=functions_info_extraction,
            function_call={"name": "extract_user_info"},
            temperature=0.2,
            max_tokens=300
        )
        payload = response.choices[0].message.get("function_call", {}).get("arguments", "{}")
        parsed = json.loads(payload)
        jsonschema.validate(instance=parsed, schema=info_schema)
        return parsed
    except Exception as e:
        print("Error:", e)
        return {}


In [ ]:
sample_chats = [
    "Hi, I'm John Doe. You can reach me at john.doe@example.com or call 9876543210. I live in Bangalore and I’m 28 years old.",
    "My name is Aisha, I’m 35, from Kochi. Email is aisha_kochi@mail.com.",
    "Hey, just wanted to ask something. I’m Rahul, no email for now, phone 9123456789, living in Delhi."
]

for i, chat in enumerate(sample_chats, start=1):
    print(f"\n=== Sample Chat {i} ===")
    print("Chat:", chat)
    extracted = extract_info_from_chat(chat)
    print("Extracted JSON:", json.dumps(extracted, indent=2))


In [ ]:
export = {
    "task1_summaries": cm.show_summaries(),
    "task2_samples": [extract_info_from_chat(c) for c in sample_chats]
}
with open("results.json", "w", encoding="utf-8") as f:
    json.dump(export, f, indent=2)
print("Results saved → results.json")
